In [ ]:
with open("input.txt", "r") as f:
# with open("example_input.txt", "r") as f:
    tiles = [list(map(int,line.strip().split(','))) for line in f if line.strip()]

In [ ]:
#part 1
max_area = 0
for i in range(len(tiles)):
    x1,y1 = tiles[i]
    for j in range(i+1, len(tiles)):
        x2,y2 = tiles[j]
        area = (abs(x1-x2)+1)*(abs(y1-y2)+1)
        max_area = max(max_area, area)

print(max_area)

In [ ]:
#part 1 but in polars
import polars as pl
df = pl.read_csv("input.txt", has_header=False, new_columns=["x", "y"]) 

# Create all pairs
pairs = df.select([
    pl.col("x").alias("x1"),
    pl.col("y").alias("y1")
]).join( 
    df.select([
        pl.col("x").alias("x2"),
        pl.col("y").alias("y2")
    ]),
    how="cross" #cross join
)

max_area = (
    pairs
    .with_columns([
        ((pl.col("x1") - pl.col("x2")).abs() + 1).alias("width"),
        ((pl.col("y1") - pl.col("y2")).abs() + 1).alias("height")
    ])
    .with_columns((pl.col("width") * pl.col("height")).alias("area"))
    .select(pl.col("area").max())
    .item() # returns a scalar, not a dataframe
)

print(max_area)

In [ ]:
#part 2
from shapely.geometry import Polygon

boundary = Polygon(tiles)
areas = []
for i in range(len(tiles)):
    x1,y1 = tiles[i]
    for j in range(i+1, len(tiles)):
        x2,y2 = tiles[j]

        min_x = min(x1,x2)
        min_y = min(y1,y2)
        max_x = max(x1,x2)
        max_y = max(y1,y2)
    
        # print(f"{x1,y1} : {(min_x2, min_y2), (min_x2, max_y2), (max_x2, max_y2), (max_x2, min_y2)}")
        rectangle = Polygon(((min_x, min_y), (min_x, max_y), (max_x, max_y), (max_x, min_y)))
        if boundary.contains(rectangle):
            areas.append((abs(min_x-max_x)+1)*(abs(min_y-max_y)+1))
            
print(max(areas))